This is the implementation of Sadamori's Node2Vec in https://github.com/skojaku/community-detection-via-neural-embedding/tree/master/libs/embcom/embcom
in one notebook, so that I can modify the functions easily.

A problem with modifying the distance metric of Node2Vec is that it uses Gensim's skipgram model (which is also highly optimized) and it doesn't allow to easily modify the distances while still keeping the optimizations (at least I haven't figured it yet).

In this notebook, we can look under the hood in an effort to modify it.

I'm running it in a different environment so that it doesn't clash with existing gensim, I will try to add functionality to quickly switch between original operation and our modifications.

In [1]:
# %% [code]
%load_ext autoreload
%autoreload 2
    

In [2]:
# -*- coding: utf-8 -*-
# @Author: Sadamori Kojaku
# @Date:   2023-06-08 16:38:19
# @Last Modified by:   Sadamori Kojaku
# @Last Modified time: 2023-06-15 11:56:36
import networkx as nx
import numpy as np
from scipy import sparse
import umap
import matplotlib.pyplot as plt


def to_trans_mat(mat):
    """
    Computes the transition matrix of a given Markov chain represented as a square matrix.

    Parameters
    ----------
    mat : numpy.ndarray
        The input matrix, which represents the transition probabilities of the Markov chain.

    Returns
    -------
    scipy.sparse.csr_matrix
        The sparse CSR matrix representing the transition matrix of the input Markov chain.
    """
    denom = np.array(mat.sum(axis=1)).reshape(-1).astype(float)
    return sparse.diags(1.0 / np.maximum(denom, 1e-32), format="csr") @ mat
 

def to_adjacency_matrix(net):
    """
    Converts an input graph representation to its corresponding adjacency matrix.

    Parameters
    ----------
    net : {scipy.sparse.csr_matrix, networkx.classes.graph.Graph, numpy.ndarray}
        The input graph representation. If it's a numpy.ndarray, it will be converted to a sparse CSR matrix.

    Returns
    -------
    scipy.sparse.csr_matrix
        The sparse CSR matrix representing the adjacency matrix of the input graph.
    """
    if sparse.issparse(net):
        if type(net) == "scipy.sparse.csr.csr_matrix":
            return net.astype(float)
        return sparse.csr_matrix(net).astype(float)
    elif "networkx" in "%s" % type(net):
        return nx.adjacency_matrix(net).astype(float)
    elif "numpy.ndarray" == type(net):
        return sparse.csr_matrix(net).astype(float)

def matrix_sum_power(A, T):
    """
    Computes the sum of powers of an input matrix, up to a given exponent.

    Parameters
    ----------
    A : numpy.ndarray
        The input square matrix to exponentiate and sum.
    T : int
        The maximum exponent to compute the sum up to.

    Returns
    -------
    numpy.ndarray
        The resulting matrix obtained by computing the sum of powers of the input matrix A, up to exponent T.
    """
    At = np.eye(A.shape[0])
    As = np.zeros((A.shape[0], A.shape[0]))
    for _ in range(T):
        At = A @ At
        As += At
    return As

/nobackup/gogandhi/miniconda3/envs/gensim_mod_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
"""This module contains sampler class which generates a sequence of nodes from
a network using a random walk.

All samplers should be the subclass of NodeSampler class.
"""
from abc import ABCMeta, abstractmethod
from numba import njit
import numpy as np

class NodeSampler(metaclass=ABCMeta):
    """Super class for node sampler class.

    Implement
        - sampling

    Optional
        - get_decomposed_trans_matrix
    """

    @abstractmethod
    def sampling(self):
        """Generate a sequence of walks over the network.

        Return
        ------
        walks : numpy.ndarray (number_of_walks, number_of_steps)
            Each row indicates a trajectory of a random walker
            walk[i,j] indicates the jth step for walker i.
        """

class Node2VecWalkSampler(NodeSampler):
    def __init__(
        self,
        num_walks=10,
        walk_length=80,
        p=1.0,
        q=1.0,
        **params
    ):
        """Noe2VecWalk Sampler

        Parameters
        ----------
        num_walks : int (Optional; Default 1)
            Number of walkers to simulate for each randomized network.
            A larger value removes the bias better but takes more time.
        walk_length : int
            Number of steps for a single walker
        p : float
            Parameter for the node2vec
        q : float
            Parameter for the node2vec
        window_length : int
            Size of the window
        verbose : bool
            Set true to display the progress (NOT IMPLEMENTED)
        """
        self.num_nodes = -1

        # parameters for random walker
        self.num_walks = int(num_walks)
        self.walk_length = walk_length
        self.p = p
        self.q = q
        self.walks = None

    def sampling(self, net):
        self.num_nodes = net.shape[0]
        self.A = net

        self.walks = simulate_node2vec_walk(
            A = self.A,
            num_walks = self.num_walks,
            walk_length = self.walk_length,
            start_node_ids = None,
            p = self.p,
            q = self.q,
        )

#
# SimpleWalk Sampler
#
class SimpleWalkSampler(Node2VecWalkSampler):
    def __init__(
        self,
        **params
    ):
        """Simple walk without bias

        Parameters
        ----------
        num_walks : int (Optional; Default 1)
            Number of walkers to simulate for each randomized network.
            A larger value removes the bias better but takes more time.
        walk_length : int
            Number of steps for a single walker
        window_length : int
            Size of the window
        verbose : bool
            Set true to display the progress (NOT IMPLEMENTED)
        """
        params["p"] = 1
        params["q"] = 1
        Node2VecWalkSampler.__init__(self, **params)


#
# node2vec walk
#
def simulate_node2vec_walk(
    A,
    num_walks,
    walk_length,
    start_node_ids = None,
    p = 1,
    q = 1,
):
    if A.getformat() != "csr":
        raise TypeError("A should be in the scipy.sparse.csc_matrix")

    if start_node_ids is None:
        start_node_ids = np.arange(A.shape[0])

    is_weighted = np.max(A.data) != np.min(A.data)
    A.sort_indices()
    if is_weighted:
        data = A.data / A.sum(axis=1).A1.repeat(np.diff(A.indptr))
        A.data = _csr_row_cumsum(A.indptr, data)

    walks = []
    for _ in range(num_walks):
        for start in start_node_ids:
            if is_weighted:
                walk = _random_walk_weighted(
                    A.indptr, A.indices, A.data, walk_length, p, q, start
                )
            else:
                walk = _random_walk(A.indptr, A.indices, walk_length, p, q, start)
            walks.append(walk.tolist())

    return walks

@njit(nogil=True)
def _csr_row_cumsum(indptr, data):
    out = np.empty_like(data)
    for i in range(len(indptr) - 1):
        acc = 0
        for j in range(indptr[i], indptr[i + 1]):
            acc += data[j]
            out[j] = acc
        out[j] = 1.0
    return out

@njit(nogil=True)
def _neighbors(indptr, indices_or_data, t):
    return indices_or_data[indptr[t] : indptr[t + 1]]


@njit(nogil=True)
def _isin_sorted(a, x):
    return a[np.searchsorted(a, x)] == x


@njit(nogil=True)
def _random_walk(indptr, indices, walk_length, p, q, t):
    max_prob = max(1 / p, 1, 1 / q)
    prob_0 = 1 / p / max_prob
    prob_1 = 1 / max_prob
    prob_2 = 1 / q / max_prob

    walk = np.empty(walk_length, dtype=indices.dtype)
    walk[0] = t
    neighbors = _neighbors(indptr, indices, t)
    if not neighbors.size:
        return walk[:1]
    walk[1] = np.random.choice(_neighbors(indptr, indices, t))
    for j in range(2, walk_length):
        neighbors = _neighbors(indptr, indices, walk[j - 1])
        if not neighbors.size:
            return walk[:j]
        if p == q == 1:
            # faster version
            walk[j] = np.random.choice(neighbors)
            continue
        while True:
            new_node = np.random.choice(neighbors)
            r = np.random.rand()
            if new_node == walk[j - 2]:
                if r < prob_0:
                    break
            elif _isin_sorted(_neighbors(indptr, indices, walk[j - 2]), new_node):
                if r < prob_1:
                    break
            elif r < prob_2:
                break
        walk[j] = new_node
    return walk


@njit(nogil=True)
def _random_walk_weighted(indptr, indices, data, walk_length, p, q, t):
    max_prob = max(1 / p, 1, 1 / q)
    prob_0 = 1 / p / max_prob
    prob_1 = 1 / max_prob
    prob_2 = 1 / q / max_prob

    walk = np.empty(walk_length, dtype=indices.dtype)
    walk[0] = t
    neighbors = _neighbors(indptr, indices, t)
    if not neighbors.size:
        return walk[:1]
    walk[1] = _neighbors(indptr, indices, t)[
        np.searchsorted(_neighbors(indptr, data, t), np.random.rand())
    ]
    for j in range(2, walk_length):
        neighbors = _neighbors(indptr, indices, walk[j - 1])
        if not neighbors.size:
            return walk[:j]
        neighbors_p = _neighbors(indptr, data, walk[j - 1])
        if p == q == 1:
            # faster version
            walk[j] = neighbors[np.searchsorted(neighbors_p, np.random.rand())]
            continue
        while True:
            new_node = neighbors[np.searchsorted(neighbors_p, np.random.rand())]
            r = np.random.rand()
            if new_node == walk[j - 2]:
                if r < prob_0:
                    break
            elif _isin_sorted(_neighbors(indptr, indices, walk[j - 2]), new_node):
                if r < prob_1:
                    break
            elif r < prob_2:
                break
        walk[j] = new_node
    return walk



In [14]:
# -*- coding: utf-8 -*-
# @Author: Sadamori Kojaku
# @Date:   2022-08-26 09:51:23
# @Last Modified by:   Sadamori Kojaku
# @Last Modified time: 2023-02-19 15:33:48
"""Module for embedding."""
# %%
import gensim
import networkx as nx
import numpy as np
from scipy import sparse
from sklearn.decomposition import TruncatedSVD


 

# Base class
#
class NodeEmbeddings:
    """Super class for node embedding class."""

    def __init__(self):
        self.in_vec = None
        self.out_vec = None

    def fit(self):
        """Estimating the parameters for embedding."""
        pass

    def transform(self, dim, return_out_vector=False):
        """Compute the coordinates of nodes in the embedding space of the
        prescribed dimensions."""
        # Update the in-vector and out-vector if
        # (i) this is the first to compute the vectors or
        # (ii) the dimension is different from that for the previous call of transform function
        if self.out_vec is None:
            self.update_embedding(dim)
        elif self.out_vec.shape[1] != dim:
            self.update_embedding(dim)
        return self.out_vec if return_out_vector else self.in_vec

    def update_embedding(self, dim):
        """Update embedding."""
        pass


class Node2Vec(NodeEmbeddings):
    """A python class for the node2vec.

    Parameters
    ----------
    num_walks : int (optional, default 10)
        Number of walks per node
    walk_length : int (optional, default 40)
        Length of walks
    window_length : int (optional, default 10)
        Restart probability of a random walker.
    p : node2vec parameter (TODO: Write doc)
    q : node2vec parameter (TODO: Write doc)
    """

    def __init__(
        self,
        num_walks=10,
        walk_length=80,
        window_length=10,
        p=1.0,
        q=1.0,
        verbose=False,
    ):
        self.in_vec = None  # In-vector
        self.out_vec = None  # Out-vector
        self.window_length = window_length
        self.sampler = Node2VecWalkSampler(
            num_walks=num_walks,
            walk_length=walk_length,
            p=p,
            q=q,
        )
        
        self.sentences = None
        self.model = None
        self.verbose = verbose

        self.w2vparams = {
            "sg": 1,
            "min_count": 0,
            "epochs": 1,
            "workers": 4,
        }

    def fit(self, net):
        """Estimating the parameters for embedding.

        Parameters
        ---------
        net : nx.Graph object
            Network to be embedded. The graph type can be anything if
            the graph type is supported for the node samplers.

        Return
        ------
        self : Node2Vec
        """
        A = to_adjacency_matrix(net)
        self.sampler.sampling(A)
        return self

    def update_embedding(self, dim):
        # Update the dimension and train the model
        # Sample the sequence of nodes using a random walk
        self.w2vparams["window"] = self.window_length

        self.w2vparams["vector_size"] = dim
        self.model = gensim.models.Word2Vec(
            sentences=self.sampler.walks, **self.w2vparams
        ) 
        #!TODO: GENSIM USES DOT SIMILARITY

        num_nodes = len(self.model.wv.index_to_key)
        self.in_vec = np.zeros((num_nodes, dim))
        self.out_vec = np.zeros((num_nodes, dim))
        for i in range(num_nodes):
            if i not in self.model.wv:
                continue
            self.in_vec[i, :] = self.model.wv[i]
            self.out_vec[i, :] = self.model.syn1neg[self.model.wv.key_to_index[i]]



class Node2VecMatrixFactorization(NodeEmbeddings):
    def __init__(self, verbose=False, window_length=10, num_blocks=500):
        self.in_vec = None  # In-vector
        self.out_vec = None  # Out-vector
        self.window_length = window_length
        self.num_blocks = num_blocks

    def fit(self, net):
        A = to_adjacency_matrix(net)

        self.A = A
        self.deg = np.array(A.sum(axis=1)).reshape(-1)
        return self

    def update_embedding(self, dim):

        P = to_trans_mat(self.A)
        Ppow = matrix_sum_power(P, self.window_length) / self.window_length
        stationary_prob = self.deg / np.sum(self.deg)
        R = np.log(Ppow @ np.diag(1 / stationary_prob))

        # u, s, v = rsvd.rSVD(R, dim=dim)
        svd = TruncatedSVD(n_components=dim + 1, n_iter=7, random_state=42)
        u = svd.fit_transform(R)
        s = svd.singular_values_
        self.in_vec = u @ sparse.diags(np.sqrt(s))
        self.out_vec = None

In [15]:
G = nx.karate_club_graph()

In [16]:
# Instantiate our Node2Vec model with custom training parameters.
node2vec_model = Node2Vec(num_walks=10, walk_length=80, window_length=10)

# Fit the model on the graph.
node2vec_model.fit(to_adjacency_matrix(G))

# Compute node embeddings with target dimension, e.g., 128.
emb_r = node2vec_model.transform(dim=64)



# # Apply UMAP to reduce dimensionality to 2D
# umap_reducer = umap.UMAP(n_components=2)
# embeddings_2d = umap_reducer.fit_transform(emb_r)

# # Plot the embeddings in 2D
# plt.figure(figsize=(8, 6))
# plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c='blue', alpha=0.6, edgecolors='k')
# plt.xlabel("UMAP Dimension 1")
# plt.ylabel("UMAP Dimension 2")
# plt.title("64D Embeddings Projected into 2D using UMAP")
# plt.show()

In [9]:
import embcom


def create_embedding(G, emb_params = {
                                            "window_length": 10,
                                            "walk_length": 80,
                                            "num_walks": 10,
                                            "dim" : 64,
                                        }):
  
    model = embcom.embeddings.Node2Vec(window_length = emb_params['window_length'], walk_length=emb_params['walk_length'], num_walks=emb_params['num_walks'])
            

    model.fit(to_adjacency_matrix(G))
    emb = model.transform(dim=emb_params['dim'])

    return emb

emb_p = create_embedding(G)

Ignore this message if you do not use Glove. Otherwise, install glove python package by 'pip install glove_python_binary' 


Check to see if the embeddings produced by our code and that from embcom module are similar to each other using normalized embedding loss

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


def calculate_normalized_embedding_loss(V_a, V_b):
    
    
    def center_embeddings(V):
        # Subtract the mean of each column from the corresponding entries
        return V - np.mean(V, axis=0)

    
    # Step 1: Center both embedding matrices
    V_a_centered = center_embeddings(V_a)
    V_b_centered = center_embeddings(V_b)
    
    # Step 2: Calculate cosine similarity matrices for centered embeddings
    C_a = cosine_similarity(V_a_centered)
    C_b = cosine_similarity(V_b_centered)
    
    # Step 3: Calculate the absolute differences between cosine similarities
    N = V_a.shape[0]
    loss = 0
    
    # Only sum over the upper triangular part of the matrix (i < j)
    for i in range(N):
        for j in range(i+1, N):
            loss += np.abs(C_a[i, j] - C_b[i, j])
    
    # Step 4: Normalize the loss
    normalized_loss = (2 / (N * (N - 1))) * loss
    
    return normalized_loss



loss = calculate_normalized_embedding_loss(emb_p, emb_r)
loss

0.16693919511584615

They are fine enough(ig).

## Going from dot similarity to Euclidean distance
Now as an initial step we want to modify N2V to use not just dot similarity but also Euclidean distance.
So note all the steps where it uses a distance measure first:

1. 